In [ ]:
# import the necessary packages
import pandas as pd
from scipy.signal import butter, lfilter, resample
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import misc
import torch
from torch.utils.data import Dataset, DataLoader
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import numpy as np
import os

from dataloaders.VTT_dataloader import DataSet_VTT
from model_train.preprocess import DataPreprocess
from models.TinyHAREncoderDecoder import TinyHAR_Encoder_Decoder_Model

pd.options.display.float_format = '{:.2f}'.format

In [ ]:

reference_point =
window_size =
step_size =
imu_position =
learning_rate =
batch_size =
num_epochs =
loss_function =

pd.options.display.float_format = '{:.2f}'.format

# check if cuda is available
print(torch.cuda.is_available())
torch.autograd.set_detect_anomaly(True)

data_path = '../data/VTT_ConIot_Dataset'
IMU_path = data_path + '/IMU'
Keypoint_path = data_path + '/Keypoint'
activities = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
users = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
num_input_features = 36
num_output_features = 3

In [ ]:
preprocess = DataPreprocess(reference_point, window_size, step_size, imu_position)
# get the processed data
keypoint_data, imu_data, sliding_windows = preprocess.processed_data()

# replace any nan value as the mean of the value before and after it
keypoint_data = keypoint_data.fillna(keypoint_data.mean())
imu_data = imu_data.fillna(imu_data.mean())

dataset_train = DataSet_VTT(keypoint_data, imu_data, sliding_windows, 1, flag='train')

train_loader = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = (window_size, num_input_features)
output_size = (window_size, num_output_features)
model = TinyHAR_Encoder_Decoder_Model(input_size, 5)
model = model.double().cuda()
if loss_function == 'cross_entropy':
    criterion = nn.CrossEntropyLoss(reduction="mean").to(device)
else:
    criterion = nn.MSELoss(reduction="mean").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
#
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# input_size = (window_size, num_input_features)
# output_size = (window_size, num_output_features)
# model = EncoderDecoder(input_size, output_size)
# model = model.double().cuda()
# if loss_function == 'cross_entropy':
#     criterion = nn.CrossEntropyLoss(reduction="mean").to(device)
# else:
#     criterion = nn.MSELoss(reduction="mean").to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#
# for epoch in range(num_epochs):
#     print(f'Epoch: {epoch}')
#     for i, data in enumerate(train_loader):
#         keypoint, imu = data
#         keypoint = keypoint.double().to(device)
#         # check if keypoint has any nan values
#         # print(torch.isnan(keypoint).any())
#         imu = imu.double().to(device)
#         output = model(keypoint)
#         output = output.squeeze(1)
#         loss = criterion(output, imu)
#
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         if i % 100 == 0:
#             print(f'Epoch: {epoch}, Loss: {loss.item()}')

